In [1]:
import ipywidgets

text_widget = ipywidgets.Textarea(
    value="Solomonoff's theory of inductive inference proposes that all problems of logical induction can be interpreted as finding a model that predicts what comes next given some sequence, and that the theoretically most likely model for what comes next should be the smallest possible computer program that outputs the sequence so far.",
    placeholder="Type something",
    description="Text to process:",
    disabled=False,
)
display(text_widget)

Textarea(value="Solomonoff's theory of inductive inference proposes that all problems of logical induction can…

In [6]:
import transformers
import torch
import tqdm
torch.manual_seed(42)  # Set a fixed seed for reproducibility

print(f"Optimizing: {text_widget.value}")

tokenizer: transformers.PreTrainedTokenizer = (
    transformers.AutoTokenizer.from_pretrained("openai-community/gpt2")
)
model: transformers.PreTrainedModel = transformers.AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2"
)

# Tokenize the text
tokens = tokenizer(
    text_widget.value,
    return_tensors="pt",
    padding=False,
    truncation=True,
    add_special_tokens=True,
)

embeddings = model.transformer.wte(tokens["input_ids"]).detach()

# We'll use a soft prompt - a special token and its hidden states for the first token that we'll use to condition the model. We'll optimize the hidden states of this token to maximize the likelihood of the text that follows it.
# Generate a soft prompt by creating random hidden states for the first token
# We'll optimize these hidden states to maximize the likelihood of the text that follows it
# past_key_values (Tuple[Tuple[torch.Tensor]] of length config.n_layers) — Contains precomputed hidden-states (key and values in the attention blocks) as computed by the model (see past_key_values output below). Can be used to speed up sequential decoding. The input_ids which have their past given to this model should not be passed as input_ids as they have already been computed.
# Shape: past_key_values (Tuple[Tuple[torch.Tensor]] of length config.n_layers)
# with each tuple having 2 tensors of shape (batch_size, num_heads, sequence_length, embed_size_per_head))
# The precision of the hidden states should be fp16
ss_prompt = (
    # One tensor of shape (1, 1, embed_size_per_head)
    torch.randn(
        1,
        4,
        model.config.n_embd,
        requires_grad=True,
        # dtype=torch.float16,
    )
    # for _ in range(model.config.n_layer)
)

# Copy the soft prompts since we'll optimize them and they'll change
original_ss_prompt = tuple(key.detach().clone() for key in ss_prompt)

# Define the optimizer
optimizer = torch.optim.Adam([ss_prompt], lr=1e-2)

# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Define the number of optimization steps
n_steps = 5000
patience = 100
patience_counter = 0
epsilon = 1e-1

# Freeze the model parameters
model.eval()

# Check that the soft prompt is optimizable (requires_grad=True)
for key in ss_prompt:
    assert key.requires_grad

# Disable gradient computation for the model
for param in model.parameters():
    param.requires_grad = False

best_loss = float("inf")
best_ss_prompt = None

# Optimize the soft prompt
for step in tqdm.tqdm(range(n_steps)):
    # Zero the gradients
    optimizer.zero_grad()

    # Add the optimizable SS prompt to the embeddings
    expanded_prompt = torch.cat([ss_prompt, embeddings], dim=1)

    # Generate the logits for the text
    logits: torch.Tensor = model(inputs_embeds=expanded_prompt).logits

    probs = torch.softmax(logits[:, ss_prompt.size(-2) - 1 :-1], dim=-1)

    # Compute the ranks of the input IDs, i.e. how many tokens would have been more likely than the correct one (the label, the input IDs)
    
    # Calculate the ranks by summing the probabilities of tokens with higher logits than the correct token
    ranks = torch.sum(probs > probs.gather(2, tokens["input_ids"].unsqueeze(-1)), dim=-1) + 1

    # Compute the loss
    loss = loss_fn(
        logits[:, ss_prompt.size(-2) - 1 :-1].reshape(-1, logits.size(-1)),
        tokens["input_ids"].reshape(-1),
    )

    # Backpropagate the gradients
    loss.backward()

    # Optimize the soft prompt
    optimizer.step()

    if step % 10 == 0:
        # Get the L2 norm of the difference between the original and optimized soft prompts
        l2_norm = sum(
            torch.norm(optimized - original, p=2)
            for optimized, original in zip(ss_prompt, original_ss_prompt)
        )
        print(
            f"Step {step}, Loss: {loss.item()}, L2 norm: {l2_norm.item()}, avg rank: {ranks.float().mean().item()}"
        )

    # Early stopping with patience
    if loss.item() < best_loss and loss.item() > epsilon:
        best_loss = loss.item()
        best_ss_prompt = ss_prompt.detach().clone()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping at step {step} with best loss {best_loss}")
        break

    # If the ranks are perfect (all 1), stop
    if torch.all(ranks == 1):
        print(f"Perfect ranks achieved at step {step}, stopping optimization.")
        break

Optimizing: Hey this is a test text of me, Aldan, writing some random text in this text box. Will this work? Perhaps!


  0%|          | 2/5000 [00:00<13:45,  6.05it/s]

Step 0, Loss: 5.8886637687683105, L2 norm: 0.5542519688606262, avg rank: 655.2222290039062


  0%|          | 12/5000 [00:01<10:57,  7.58it/s]

Step 10, Loss: 5.159086227416992, L2 norm: 3.46441388130188, avg rank: 439.6666564941406


...

Step 440, Loss: 0.19830304384231567, L2 norm: 29.436723709106445, avg rank: 1.0370370149612427


  9%|▉         | 452/5000 [01:05<10:41,  7.09it/s]

Step 450, Loss: 0.17161428928375244, L2 norm: 29.54309844970703, avg rank: 1.0370370149612427


  9%|▉         | 454/5000 [01:05<10:54,  6.94it/s]

Perfect ranks achieved at step 454, stopping optimization.


In [7]:
# Save the best soft prompt
torch.save(best_ss_prompt, "best_ss_prompt.pt")

In [8]:
import os
import transformers
import torch
import tqdm

if "tokenizer" not in locals():
    tokenizer: transformers.PreTrainedTokenizer = (
        transformers.AutoTokenizer.from_pretrained("openai-community/gpt2")
    )
if "model" not in locals():
    model: transformers.PreTrainedModel = transformers.AutoModelForCausalLM.from_pretrained(
        "openai-community/gpt2"
    )

# Load the best soft prompt from file
best_ss_prompt = torch.load("best_ss_prompt.pt")

# Do a greedy decoding manually
# Pass the optimized soft prompt to the model
# We can't use .generate() since we need to pass the inputs_embeds
all_logits = []
generated_tokens = []
with torch.no_grad():
    for i in tqdm.tqdm(range(150)):
        # Generate the logits for the next token using what we've generated so far
        # If there are no generated tokens yet, just take the soft prompt
        if len(generated_tokens) == 0:
            expanded_prompt = best_ss_prompt
        else:
            expanded_prompt = torch.cat(
                [
                    ss_prompt,
                    model.transformer.wte(
                        torch.tensor(generated_tokens).unsqueeze(0)
                    ).detach(),
                ],
                dim=1,
            )

        assert expanded_prompt.shape == (
            1,
            best_ss_prompt.size(-2) + len(generated_tokens),
            model.config.n_embd,
        ), f"Got size {expanded_prompt.shape} instead of (1, {best_ss_prompt.size(-2) + len(generated_tokens)}, {model.config.n_embd})"

        # Generate the logits for the text
        logits: torch.Tensor = model(inputs_embeds=expanded_prompt).logits

        # Get the logits for the next token
        next_token_logits = logits[0, -1, :]

        # Get the token with the highest probability
        next_token = next_token_logits.argmax().item()

        # Append the token and its logits
        generated_tokens.append(next_token)
        all_logits.append(next_token_logits)

        if next_token == tokenizer.eos_token_id:
            break

all_logits = torch.stack(all_logits).view(1, -1, model.config.vocab_size)
generated_tokens = torch.tensor(generated_tokens)

if "text_widget" in locals():
    reference_text = text_widget.value
    reference_tokens = tokenizer.encode(
        reference_text, add_special_tokens=False, return_tensors="pt"
    )
else:
    reference_text = ""
    reference_tokens = torch.ones_like(generated_tokens)

# Decode the generated tokens
generated_text = tokenizer.decode(generated_tokens)
print(f"Reference: {reference_text}")
print(f"Generated: {generated_text}")

# Print the generated text with the rank of each token
for index, token in enumerate(generated_tokens):
    this_tok_logits = all_logits[:, index, :]
    # How many tokens have a logit > than this token's logit
    count_higher_logits = this_tok_logits[..., :] > this_tok_logits[..., token]
    this_tok_probs = torch.softmax(this_tok_logits, dim=-1)
    try:
        reference_token = reference_tokens[0, index].item()
    except IndexError:
        reference_token = None
    print(
        f"'{tokenizer.decode(token)}':\tRank {count_higher_logits.sum().item():.2f}, probability: {this_tok_probs[..., token].item() * 100:.2f}%, Reference: '{tokenizer.decode(reference_token) if reference_token is not None else 'N/A'}'"
    )

100%|██████████| 150/150 [00:10<00:00, 14.83it/s]


Reference: Hey this is a test text of me, Aldan, writing some random text in this text box. Will this work? Perhaps!
Generated: Hey this is a test text of me, Aldan, writing some random text in this text box. Will this work? Perhaps! I'll have to test this out. Maybe. I'll have to test this out some more. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe. Maybe
'Hey':	Rank 0.00, probability: 71.02%, Reference: 'Hey'
' this':	Rank 0.00, probability: 87.04%, Reference: ' this'
' is':	Rank 0.00, probability: 99.65%, Reference: ' is'
' a':	Rank 0.00, probability: 96.23%, Reference: ' a'
' test':	Rank 0.00, probability: 90.31%, Reference: ' test'
' text':	Rank 0.00, probability: 95.40%, Ref